In [66]:
####################################################################################
#                                                                                  #                 
# Trabalho Final das Disciplinas Estatística Aplicada e Programação com R e Python #
# Discentes: Isabella Calfa e Taian Feitosa                                        #
#                                                                                  #
####################################################################################

#Libs:
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats

In [347]:
df1 = pd.read_csv("IMDb movies.csv")
df2 = pd.read_csv("IMDb names.csv")
df3 = pd.read_csv("IMDb ratings.csv")
df4 = pd.read_csv("IMDb title_principals.csv")

C:\Users\Isabella Calfa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
#Tratamento de dados df1 (movies.csv)
df1.info()

In [ ]:
#Dados df1 (movies.csv)
df1.head()
#Corrigir nomes dos países, idioma, data de publicação (tem algumas só com ano) e ano (tem não int)
#Determinar quantos países e quantos idiomas e qual o principal (primeiro)

In [537]:
#Vamos tratar as colunas "country" e "language" para reduzir o quantitativo de valores nulos.
df1 = pd.read_csv("IMDb movies.csv", low_memory=False)
#Total de linhas:
total_rows=df1["country"]
#Ajuste do nome do país:
print(f'\n********* Correção por companhia *********')
df1["country"]=df1["country"].astype(str)
df1["production_company"]=df1["production_company"].astype(str)
#Filmes sem países:
no_country=df1[df1["country"]=="nan"].count()
print(f'Filmes sem países: {no_country["imdb_title_id"]}')
#Conversão pela moda:
dfx=df1[df1["country"]!="nan"] #Retirando países NaN pós conversão para string
dfx=df1.groupby(["production_company"])["country"].agg([pd.Series.min,pd.Series.mode])
dfx["mode_string"]=dfx["mode"].astype(str).str.startswith('[') 
df1=pd.merge(df1,dfx, how='left', on="production_company")
##Se houver mais de uma moda, considera pela primeira vez que a companhia apareceu
df1.loc[(df1["country"]=="nan") & (df1["mode_string"]==False),"country"]=df1["mode"]
df1.loc[(df1["country"]=="nan") & (df1["mode_string"]==True),"country"]=df1["min"]
#Títulos de teste:
df1[(df1["imdb_title_id"]=="tt3248148") | (df1["imdb_title_id"]=="tt0000009") | (df1["imdb_title_id"]=="tt10452854")]
#Validação pós correção:
no_country=df1[df1["country"]=="nan"].count()
print(f'Filmes sem países pós correção por companhia: {no_country["imdb_title_id"]}')


********* Correção por companhia *********
Filmes sem países: 64
Filmes sem países pós correção por companhia: 26


In [445]:
#Filmes sem países:
no_country_no_company=df1[(df1["country"]=="NI") & df1["production_company"].isnull()].count()
print(f'Filmes sem países e sem companhias: {no_country_no_company["imdb_title_id"]}')
#Alterando Null para NI
df1.loc[df1["country"].isnull(),"country"]="NI"
country_null=df1[df1["country"].isnull()].count()
print(f'Filmes sem países: {country_null["imdb_title_id"]}')
country_NI=df1[df1["country"]=="NI"].count()
print(f'Filmes com país NI: {country_NI["imdb_title_id"]}')

Filmes sem países: 64
Filmes sem países: 0
Filmes com país NI: 64


In [443]:
df1[df1["production_company"]=="Big Book Media"]["country"]

70482                  NI
70500                  NI
71285    UK, USA, Morocco
73086    USA, UK, Morocco
79347                  UK
Name: country, dtype: object

In [489]:
#df1[df1["imdb_title_id"]=="tt3248148"]["production_company"]
a=df1[df1["production_company"]=="Big Book Media"]["country"].str.split(',').astype(str)
print(a)
v=[]
for i in range(0,5):
    v.append(a.iloc[i])
print(v)
#abs.map(lambda x: ','.join(sorted(x)))

70482                       ['nan']
70500                       ['nan']
71285    ['UK', ' USA', ' Morocco']
73086    ['USA', ' UK', ' Morocco']
79347                        ['UK']
Name: country, dtype: object
["['nan']", "['nan']", "['UK', ' USA', ' Morocco']", "['USA', ' UK', ' Morocco']", "['UK']"]


In [ ]:
#Quantidade de países (ou grupo de) distintos:
countries=df1["country"].apply(lambda x: str(x)).unique()
print(f'Qtd. países (ou grupo): {len(countries):,.0f}.')
print(f'Países: \n{countries[:50]}')

In [ ]:
#Definindo quantidade de países:
n_country=df1.groupby(["imdb_title_id"])[df1[df1["country"].str.contains(",")]].count()
n_country.head(5)
#Definindo país princial:

In [ ]:
#Filmes sem idioma:
no_country=df1[df1["language"].isnull()].count()
print(f'Filmes sem idioma: {no_country["imdb_title_id"]}')
no_language_no_country=df1[(df1["country"].isnull() & df1["language"].isnull())].count()
print(f'Filmes sem países e sem idioma: {no_language_no_country["imdb_title_id"]}')
#Moda do idioma por país do filme que produz: 
print(f'\n********* Correção por país *********')
#modal_language_by_country=df1.groupby(["country"])["language"].agg(pd.Series.mode).to_frame(name='new_language').reset_index()
#df1.head(2)
#df1=pd.merge(df1,modal_country_by_company, how='left', on="production_company")
#df1.loc[df1["country"].isnull(),"country"]=df1.new_country
#no_country=df1[df1["country"].isnull()].count()
#print(f'Filmes sem países pós correção por companhia: {no_country["imdb_title_id"]}')
#TRATAR IDIOMA NONE E NAN

In [ ]:
df2.info()
#Sem necessidade de tratamento dos dados
#Se for utilizar dados de datas de nascimento e morte, precisa tratar.

In [ ]:
df3.info()
#Sem necessidade de tratamento dos dados

In [ ]:
df4.info()
#Sem necessidade de tratamento dos dados

In [ ]:
df4[df4["job"].notnull()]